# <font color='blue'>Part 1: Training Data generation</font>

---

**Panctuation addition** is an NLP task, that by reciving an unpunctuated text, adds panctuation. Typically it is a task on a pipe of text to speach, i.e., the input of voice recorded sentences needs to be transformed to text. Panctuation block on the pipe recives as input chunks of words (from previous block) in size of ~150 words;<br>
Therefore we will generate chunks of that size, and will make sure that the chunks consist complete sentences (not ending in the middle of a sentence).<br>
As our task is **Hebrew** punctuation addition, we chose to get text from Wikipedia (Hebrew) articles;<br>

Data generation main blocks:<br>
1) Fetch - by reciving an address of html (wikipedia) page, copy the source code;<br>
2) Clean - clean the text <br>
3) Truncate - get only the relevant segments of code - the text<br>
4) format - replace some types of words as numeric and english to constant tokens<br>
5) Test for paragraph sufficient leanght<br>
6) Vectorize - generate 2 same leangth vectors of X - words without punctuation, and Y - the tags;<br>
7) Verify Data - checking the vectors coding OK;<br>
8) aggregate - save all in a file (only if URL was not previously processed)<br>

---

### <font color='blue'>Data generation 1: Fetch</font>

In [1]:
from urllib import request
import re

def getSingleURLSourceCode(URL):
    # get handle to URL
    response = request.urlopen(URL)
    # fetch source code
    sourceCode = response.read().decode('utf8')
    return sourceCode

targetUrlList = [("קפיטליזם","https://he.wikipedia.org/wiki/%D7%A7%D7%A4%D7%99%D7%98%D7%9C%D7%99%D7%96%D7%9D"),
                 ("קרל מרקס","https://he.wikipedia.org/wiki/%D7%A7%D7%A8%D7%9C_%D7%9E%D7%A8%D7%A7%D7%A1"),
                 ("אילן רמון","https://he.wikipedia.org/wiki/%D7%90%D7%99%D7%9C%D7%9F_%D7%A8%D7%9E%D7%95%D7%9F"),
                 ("אלברט איינשטיין","https://he.wikipedia.org/wiki/%D7%90%D7%9C%D7%91%D7%A8%D7%98_%D7%90%D7%99%D7%99%D7%A0%D7%A9%D7%98%D7%99%D7%99%D7%9F")]

# אלברט איינשטיין
sourceCode = getSingleURLSourceCode("https://he.wikipedia.org/wiki/%D7%90%D7%9C%D7%91%D7%A8%D7%98_%D7%90%D7%99%D7%99%D7%A0%D7%A9%D7%98%D7%99%D7%99%D7%9F")
# אילן רמון
# sourceCode = getSingleURLSourceCode("https://he.wikipedia.org/wiki/%D7%90%D7%99%D7%9C%D7%9F_%D7%A8%D7%9E%D7%95%D7%9F")
print(sourceCode)

<!DOCTYPE html>
<html class="client-nojs" lang="he" dir="rtl">
<head>
<meta charset="UTF-8"/>
<title>אלברט איינשטיין – ויקיפדיה</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"אלברט_איינשטיין","wgTitle":"אלברט איינשטיין","wgCurRevisionId":24977594,"wgRevisionId":24977594,"wgArticleId":83,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["ויקיפדיה: ערכים עם כותרות קישור שנוצרו על ידי בוט","ויקינתונים - השוואת ערכים: לא מתאים","ויקינתונים - השוואת ערכים: לא מתאים: תחום","ויקינתונים - השוואת ערכים: חסרה תווית עברית","ויקינתונים - השוואת ערכים: חסרה תווית עברית: מקום מגורים","ויקינתונים:ערכים חסרי תווית בעברית: מקום לימודים","ויקינתונים:ערכים חסרי תווית בעברית","וי

### <font color='blue'>Data generation 2: Clean</font>
exploring the date, we note that the relevant text lies within paragraph delimiters of HTML code, i.e.<br>
< p > for beginning of paragraph<br>
< / p > for ending of paragraph<br>
before we truncate acording to these delimiters, we note many source code syntex in the paragraph e.g. links (of some types) and bold syntax;<br>
We present the following cleanning code, that proved to keep most of the paragraphs:

In [2]:
def replaceInternalLinkWithText(inText):
    # function replaces link syntax in html code with it's shown text
    pattern = re.compile(r'<a href="[^"]*" title="[^"]*">([^<]*)</a>')
    outText = pattern.sub(r'\1', inText)
    return outText

def replaceExternalLinkWithText(inText):
    # function replaces link syntax in html code with it's shown text
    pattern = re.compile(r'<a rel="[^"]*" class="[^"]*" href="[^"]*">([^<]*)</a>')
    outText = pattern.sub(r'\1', inText)
    return outText

def replaceRedirectWithText(inText):
    # function replaces link syntax in html code with it's shown text
    pattern = re.compile(r'<a href="[^"]*" class="[^"]*" title="[^"]*">([^<]*)</a>')
    outText = pattern.sub(r'\1', inText)
    return outText

def replaceSmallReferenceLink(inText):
    # function replaces small reffences syntax in html code with it's shown text
    pattern = re.compile(r'<small class="[^"]*" dir="[^"]*">([^<]*)</small>')
    outText = pattern.sub(r'\1', inText)
    return outText

def replaceHebQuateCode(inText):
    # function replaces HTML heb quatation with shown text
    pattern = re.compile(r'<span class="[^"]*">([^<]*)</span>')
    outText = pattern.sub(r'\1', inText)
    return outText

def removeReferenceLink(inText):
    # function removes reffences from text
    pattern = re.compile(r'<sup id="[^"]*" class="[^"]*"><a href="[^"]*">[^<]*</a></sup>')
    outText = pattern.sub(r'', inText)
    return outText

def removeBold(inText):
    pattern = re.compile(r'<b>([^<]*)</b>')
    outText = pattern.sub(r'\1',inText)
    return outText

### <font color='blue'>Data generation 3: Truncate</font>
Below we truncate according to paragraph delimiters and clean HTML code from the paragraph text;<br>
We also pass a list of allowed (punctuation) symbols<br>

That stage output encapsulte stages {2,3} and the paragraph output should look as it looks in Wikipedia

In [3]:
def getParagraph(inStr: str, punctList):
    regExp =  r'<p>[א-ת0-9a-zA-Z\s' + r''.join(punctList) + r']*</p>'
    # print(regExp)
    strListWithDelimiters = re.findall(regExp, inStr, re.UNICODE)
    strListNoDelimiters = [re.sub(r'<p>', '', strWithDelimiter) for strWithDelimiter in strListWithDelimiters]
    strListNoDelimiters = [re.sub(r'</p>', '', strWithDelimiter) for strWithDelimiter in strListNoDelimiters]
    return strListNoDelimiters

punctList = [r'.', r',', r'?',r'!',r'(',r')', r'—', r'–' ,r'\-', r'־' , r'"', r':', r';', '\'', r'\[', r'\]']
print(punctList)

['.', ',', '?', '!', '(', ')', '—', '–', '\\-', '־', '"', ':', ';', "'", '\\[', '\\]']


In [4]:
def getCleanParagraph(inRawText, punctList):
    # function recives raw HTML hebrew wikipedia text and returns list of clean paragraphs
    # cleans: {reference link; internal link; external link; redirect link; bold mark}
    editedText = removeReferenceLink(inRawText)
    editedText = replaceInternalLinkWithText(editedText)
    editedText = replaceExternalLinkWithText(editedText)
    editedText = replaceRedirectWithText(editedText)
    editedText = replaceSmallReferenceLink(editedText)
    editedText = replaceHebQuateCode(editedText)
    editedText = removeBold(editedText)
    # print(editedText)
    return getParagraph(editedText, punctList)

paragraphList = getCleanParagraph(sourceCode, punctList)

for chunk in paragraphList:
    print(chunk)

איינשטיין נחשב לגדול התאורטיקנים, לצד אייזק ניוטון, אבי המכניקה הקלאסית. שמו הפך מילה נרדפת לגאונות. הוא זכה לפרסום עולמי ברבע הראשון של המאה ה-20 בזכות תורת היחסות אותה פיתח, אשר שינתה את כל מה שהיה ידוע עד אז על מהותם של הזמן, המרחב, המסה, התנועה וכוח הכבידה וכן בזכות תרומותיו לתחומי מכניקת הקוונטים והמכניקה הסטטיסטית ולהסברת האפקט הפוטואלקטרי. בגין תרומתו הייחודית בנושא האחרון הוא זכה בפרס נובל לפיזיקה בשנת 1921.

מלבד חלקו המכריע בפיתוח תחומי המדע, איינשטיין התבטא ופעל גם בנושאים אקטואליים והיה ציוני נלהב ותומך במדינת ישראל. עם זאת, הוא דחה שתי הצעות שקיבל מהמדינה הצעירה, האחת, להיות נשיא המדינה; השנייה, להיות נשיא האוניברסיטה העברית בירושלים, לה הוריש בצוואתו את כל כתביו. איינשטיין תרם גם לפרויקט פיתוח נשק גרעיני בארצות הברית, דרך תמיכתו ב-מכתב איינשטיין-סילארד לנשיא האמריקאי, ועידדו לקדם פיתוחו בארצות הברית, על אף שהגדיר עצמו כפציפיסט.

איינשטיין נולד בעיר אולם, שהייתה בממלכת וירטמברג שבקיסרות הגרמנית (כיום במדינת באדן-וירטמברג) ב-14 במרץ 1879, למשפחה יהודית – פאולינה לבית קוך וה

### <font color='blue'>Data generation 4: Format words</font>
Adding constants replacments for english and numeric words. e.g.<br>
'987,654,321.00' => '\<מספר\>'<br>
'F-16' => '\<אנגלית\>'<br>

Stage 4 functionality:
* replace english words with a constant string: \<אנגלית\> <br>
* replace digits with a constant string: \<מספר\><br>
* all punctuation remains.

**TBD - consult with michael:**
* consider: since the < > charachters are recognized by the nltk tokenizer, can use only א-ת words for the cnstants insted; <br>is that OK? <br>we will have the pre-words attached as a problem; במספר insted of ב מספר<br>
=> will use 0 for all numbers
=> will use ENG for all english
* consider: all dash types to a single one, with spaces <br>
=> removed
* consider: replace all remained punctuation apart of ',', '.'<br>
=> kept also '?', '!'

In [5]:
def replaceEnglish(inText, punctList, englishString = '<אנגלית>'):
    # function replaces 
    punct = r''.join(punctList)
    pattern = re.compile(r'([a-zA-Z0-9]+[' + punct + r']*[a-zA-Z]+[a-zA-Z0-9]*|[a-zA-Z0-9]*[a-zA-Z]+[' + punct + r']*[a-zA-Z0-9]+|[a-zA-Z0-9]+[' + punct + r']*[a-zA-Z0-9]*[a-zA-Z]+|[a-zA-Z]+[a-zA-Z0-9]*[' + punct + r']*[a-zA-Z0-9]+)')
    #pattern = re.compile(r'([a-zA-Z0-9]+[' + punct + r']*[a-zA-Z]+[a-zA-Z0-9]*|[a-zA-Z0-9]*[a-zA-Z]+[' + punct + r']*[a-zA-Z0-9]+)')
    outText = pattern.sub(" " + englishString + " ", inText)
    pattern = re.compile(r'[a-zA-Z]')
    outText = pattern.sub(" " + englishString + " ", outText)
    return outText

def replaceNumber(inText, numberString = '<מספר>'):
    # function replaces 
    pattern = re.compile(r'([0-9]+[,])*[0-9]+(\.[0-9]+)?')
    outText = pattern.sub(" " + numberString + " ", inText)
    return outText

In [6]:
def formatParagraph(inParagraph):
    # function recives paragraph, and return formatted text
    # outParagraph = replaceEnglish(inParagraph, punctList, '<אנגלית>')
    outParagraph = replaceEnglish(inParagraph, punctList, 'גבולאנגליתלובג')
    # outParagraph = replaceNumber(outParagraph, '<מספר>')
    outParagraph = replaceNumber(outParagraph, 'גבולמספרלובג')
    return outParagraph

formattedParagraphList = [formatParagraph(paragraph) for paragraph in paragraphList]

for chunk in formattedParagraphList:
    print(chunk)

איינשטיין נחשב לגדול התאורטיקנים, לצד אייזק ניוטון, אבי המכניקה הקלאסית. שמו הפך מילה נרדפת לגאונות. הוא זכה לפרסום עולמי ברבע הראשון של המאה ה- גבולמספרלובג  בזכות תורת היחסות אותה פיתח, אשר שינתה את כל מה שהיה ידוע עד אז על מהותם של הזמן, המרחב, המסה, התנועה וכוח הכבידה וכן בזכות תרומותיו לתחומי מכניקת הקוונטים והמכניקה הסטטיסטית ולהסברת האפקט הפוטואלקטרי. בגין תרומתו הייחודית בנושא האחרון הוא זכה בפרס נובל לפיזיקה בשנת  גבולמספרלובג .

מלבד חלקו המכריע בפיתוח תחומי המדע, איינשטיין התבטא ופעל גם בנושאים אקטואליים והיה ציוני נלהב ותומך במדינת ישראל. עם זאת, הוא דחה שתי הצעות שקיבל מהמדינה הצעירה, האחת, להיות נשיא המדינה; השנייה, להיות נשיא האוניברסיטה העברית בירושלים, לה הוריש בצוואתו את כל כתביו. איינשטיין תרם גם לפרויקט פיתוח נשק גרעיני בארצות הברית, דרך תמיכתו ב-מכתב איינשטיין-סילארד לנשיא האמריקאי, ועידדו לקדם פיתוחו בארצות הברית, על אף שהגדיר עצמו כפציפיסט.

איינשטיין נולד בעיר אולם, שהייתה בממלכת וירטמברג שבקיסרות הגרמנית (כיום במדינת באדן-וירטמברג) ב- גבולמספרלובג  במרץ  גבולמס

### <font color='blue'>Data generation 5: Test for length</font>


#### <font color='blue'>Data generation 5.1: Properties of generated data relative to task typical input data:</font>
Text to Speach input data is known to be seperated according to long pauses, typically holding ~150 words; Each such set of words is reffered to as 'chunk'; We assume that each such chunk consist of some general subject, and hope that the context within can help in out panctuation taskl; Therefore we strive to maintain that per-paragraph-context in out syntetic data as well.<br>

We note that above data is with variant word size per paragraph, but we wil not concetenate and divide above paragraph to chunks sized ~150 as we may lose the context-per-wikipedia-paragraph that we have now.<br>

**Conclusion:** Paragraph size will remain as got from Wikipedia;<br>

**Q. what can we do to adjust synthetic data to typical tast data?<br>**
A. We may select large paragraphs to train on.<br>

We now incoporate above desicion; we will keep only paragraphs of size $>50$ words

In [7]:
import nltk
from nltk import word_tokenize, sent_tokenize
minNumOfWords = 50

def erasePunctuation(tokenList):
    newList = []
    for token in tokenList:
        if not(re.match("^\W+$", token)):
            newList.append(token)
    return newList

def keepLongParagraphs(paragraphList, minNumOfWords = 70):
    outParagraphList = []
    for paragraph in paragraphList:
        tokenList =  nltk.word_tokenize(paragraph)
        noPunctTokenList = erasePunctuation(tokenList)
        paragraphLength = len(tokenList)
        numOfWords = len(noPunctTokenList)
        if (numOfWords >= minNumOfWords):
            # here can insert more parameters of paragraph e.g. num of english words, num of digit words etc.
            outParagraphList.append((paragraph, numOfWords))
    return outParagraphList

longParagraphList = keepLongParagraphs(formattedParagraphList, minNumOfWords)
print([paragraph for (paragraph, numOfWords) in longParagraphList])
print("removed %d short paragraphs" %(len(paragraphList) - len(longParagraphList)))

['איינשטיין נחשב לגדול התאורטיקנים, לצד אייזק ניוטון, אבי המכניקה הקלאסית. שמו הפך מילה נרדפת לגאונות. הוא זכה לפרסום עולמי ברבע הראשון של המאה ה- גבולמספרלובג  בזכות תורת היחסות אותה פיתח, אשר שינתה את כל מה שהיה ידוע עד אז על מהותם של הזמן, המרחב, המסה, התנועה וכוח הכבידה וכן בזכות תרומותיו לתחומי מכניקת הקוונטים והמכניקה הסטטיסטית ולהסברת האפקט הפוטואלקטרי. בגין תרומתו הייחודית בנושא האחרון הוא זכה בפרס נובל לפיזיקה בשנת  גבולמספרלובג .\n', 'מלבד חלקו המכריע בפיתוח תחומי המדע, איינשטיין התבטא ופעל גם בנושאים אקטואליים והיה ציוני נלהב ותומך במדינת ישראל. עם זאת, הוא דחה שתי הצעות שקיבל מהמדינה הצעירה, האחת, להיות נשיא המדינה; השנייה, להיות נשיא האוניברסיטה העברית בירושלים, לה הוריש בצוואתו את כל כתביו. איינשטיין תרם גם לפרויקט פיתוח נשק גרעיני בארצות הברית, דרך תמיכתו ב-מכתב איינשטיין-סילארד לנשיא האמריקאי, ועידדו לקדם פיתוחו בארצות הברית, על אף שהגדיר עצמו כפציפיסט.\n', 'איינשטיין נולד בעיר אולם, שהייתה בממלכת וירטמברג שבקיסרות הגרמנית (כיום במדינת באדן-וירטמברג) ב- גבולמספרלובג  במ

---

### <font color='blue'>Data generation 6: Vectorize</font>
At this point we have punctuated paragraphs (the paragraphs are in good size, as the wanted input chunks), We would like to remove non relevant punctuation, generate tag vector per paragraph, and concetenate all with delimiters;<br>
* puntuation remuval:<br>
At this point we **choose** to remove all punctuations apart of {'.',',','!','?'}; That is due to:<br>
    * some double punctuations e.g. sequence of '.' and ')' in the end of a sentence<br>
    * in-word puctuaiton (e.g. abrevations, and dash)<br>
* Vectorize:<br> 
    * generate vectors of tags (containing the remaining punctuations tags)
    * adding delimiter betweed words vector and tags vector: < XYDEL >
* concetenate
    * add delimiter between taged paragraphs < DATADEL ><br>

In [8]:
keepPuncList = [r'.', r',', r'?',r'!']
removePunctList = [r'(',r')', r'—', r'–' ,r'\-', r'־' , r'"', r':', r';', '\'', r'\[', r'\]']
numConst            = r'0'
engConst            =  r'ENG'
XY_DelConst         = r'<XYDEL>'

def removePunct(inText, removePunctList):
    punct = r''.join(removePunctList)
    pattern = re.compile(r'[' + punct + ']')
    outText = pattern.sub(r' ',inText)
    return outText

def relpaceConstStr(inStr, matchStr, replaceStr):
    pattern = re.compile(matchStr)
    outStr = pattern.sub(replaceStr,inStr)
    return outStr
    
def tag_paragraph(paragraph, punctList):
    tokens = nltk.tokenize.word_tokenize(paragraph)
    x = []
    y = []
    for w in tokens:
        if w in punctList:
            y.pop()
            y.append(w)
        else:
            y.append("none")
            x.append(w)
    # for word, tag in zip(x,y): print("%s %s" %(word, tag))
    return x,y

def getXY(paragraph, removePunctList, keepPuncList, inNumConst = numConst, inEngConst = engConst, inXY_DelConst = XY_DelConst):
    # remove unwanted punctuation
    noPunctParagraph = removePunct(paragraph, removePunctList)

    # tag paragraph with remain punctuation
    X_vec, Y_vec = tag_paragraph(noPunctParagraph, keepPuncList)

    # joinn to string
    X_str = " ".join(X_vec)
    X_str = relpaceConstStr(X_str, "גבולמספרלובג", inNumConst)
    X_str = relpaceConstStr(X_str, "גבולאנגליתלובג", inEngConst)
    Y_str = " ".join(Y_vec)
    #  here can add record info
    XY_str = X_str + inXY_DelConst + Y_str
    return XY_str

# paragraph = longParagraphList[0]
# print(getXY(paragraph[0], removePunctList, keepPuncList))
XY_List = [getXY(paragraph[0], removePunctList, keepPuncList) for paragraph in longParagraphList]
# for XY in XY_List: print(XY)

DATA = "<DATADEL>".join(XY_List)
print(DATA)

איינשטיין נחשב לגדול התאורטיקנים לצד אייזק ניוטון אבי המכניקה הקלאסית שמו הפך מילה נרדפת לגאונות הוא זכה לפרסום עולמי ברבע הראשון של המאה ה 0 בזכות תורת היחסות אותה פיתח אשר שינתה את כל מה שהיה ידוע עד אז על מהותם של הזמן המרחב המסה התנועה וכוח הכבידה וכן בזכות תרומותיו לתחומי מכניקת הקוונטים והמכניקה הסטטיסטית ולהסברת האפקט הפוטואלקטרי בגין תרומתו הייחודית בנושא האחרון הוא זכה בפרס נובל לפיזיקה בשנת 0<XYDEL>none none none , none none , none none . none none none none . none none none none none none none none none none none none none none , none none none none none none none none none none none none , , , none none none none none none none none none none none none none . none none none none none none none none none none none .<DATADEL>מלבד חלקו המכריע בפיתוח תחומי המדע איינשטיין התבטא ופעל גם בנושאים אקטואליים והיה ציוני נלהב ותומך במדינת ישראל עם זאת הוא דחה שתי הצעות שקיבל מהמדינה הצעירה האחת להיות נשיא המדינה השנייה להיות נשיא האוניברסיטה העברית בירושלים לה הוריש בצוואתו את כל כתביו

### <font color='blue'>Data generation 7: Verify Data</font>
TBD: consider ommit this part;<br>
We add data verification stage to eliminate bad data e.g. empty pargraph;
We cought a case where an entire URL has failed to produce a single paragraph, therefore we had empty vector wich errored our run. we introduce vector XY check that may be upgraded when needed;<br>
No empty paragraph, actually found empty URL (preprocess pipe OK);

In [9]:
def checkVectorOK(inVector):
    # vector check function
    vectorOk = True
    if (inVector == ""): vectorOk = False
    if (len(inVector)<minNumOfWords): vectorOk = False
    return vectorOk

def getValidVectorList(inXYList):
    outXYList = []
    for xy in inXYList:
        if checkVectorOK(xy): outXYList.append(xy)
        else: print("print: cought bad vector = %s" %xy)
    return outXYList

### <font color='blue'>Data generation 8: Agregate</font>
Plugging all togather to single method that recives URL adress and returns a data in above format

In [10]:
DATA_DelConst       = r'<DATADEL>'

def getDataFromURL(URL, inNumConst = numConst, inEngConst = engConst, inXY_DelConst = XY_DelConst, inDATA_DelConst = DATA_DelConst):
    # fetch
    sourceCode = getSingleURLSourceCode(URL)

    # clean and truncate
    paragraphList = getCleanParagraph(sourceCode, punctList)

    # format words
    formattedParagraphList = [formatParagraph(paragraph) for paragraph in paragraphList]

    # keep long paragraphs only
    longParagraphList = keepLongParagraphs(formattedParagraphList, minNumOfWords)

    # vectorize
    XY_List = [getXY(paragraph[0], removePunctList, keepPuncList, inNumConst, inEngConst, inXY_DelConst) for paragraph in longParagraphList]
    
    # check vectors 
    # XY_Valid_List = getValidVectorList(XY_List)
    XY_Valid_List = XY_List
    
    # aggregate
    DATA = inDATA_DelConst.join(XY_Valid_List)
    return DATA

# Data = getDataFromURL("https://he.wikipedia.org/wiki/%D7%90%D7%99%D7%9C%D7%9F_%D7%A8%D7%9E%D7%95%D7%9F")
# print(Data)

def getDataFromURLList(URLList):
    dataList = [getDataFromURL(URL) for URL in URLList]
    outData = DATA_DelConst.join(dataList)
    return outData

    
targetUrlList = ["https://he.wikipedia.org/wiki/%D7%A7%D7%A4%D7%99%D7%98%D7%9C%D7%99%D7%96%D7%9D",
                 "https://he.wikipedia.org/wiki/%D7%A7%D7%A8%D7%9C_%D7%9E%D7%A8%D7%A7%D7%A1",
                 "https://he.wikipedia.org/wiki/%D7%90%D7%99%D7%9C%D7%9F_%D7%A8%D7%9E%D7%95%D7%9F",
                 "https://he.wikipedia.org/wiki/%D7%90%D7%9C%D7%91%D7%A8%D7%98_%D7%90%D7%99%D7%99%D7%A0%D7%A9%D7%98%D7%99%D7%99%D7%9F"]

outData = getDataFromURLList(targetUrlList)
print(outData)

השימוש הראשון המוכר במונח קפיטליזם מופיע אצל תאקרי בשנת 0 במשמעות בעלות על הון השימוש המודרני הרחב הראשון במונח בא בספר הקפיטל של קרל מרקס ופרידריך אנגלס המתייחס למערכת הייצור הקפיטליסטית ול קפיטליסט במובן של בעלים של אמצעי הייצור שימושם המקורי של מרקס ואנגלס במונח היה ככינוי גנאי מאקס ובר השתמש בו בחיבורו הידוע האתיקה הפרוטסטנטית ורוח הקפיטליזם אך רק לאחר פרסום ספרו של הכלכלן ורנר זומברט קפיטליזם מודרני משנת 0 המונח הפך לתיאור תפיסה המנוגדת לסוציאליזם כיום אף שאדם סמית לא השתמש מעולם במונח קפיטליזם ותיאר את המערכת הכלכלית המועדפת עליו כ מערכת החירות הטבעית יש הרואים בו אבי הקפיטליזם<XYDEL>none none none none none none none none none , none none none . none none none none none none none none none none none none , none none none none none none none none none none none . none none none none none none none none . none none none none none none none none none none none none none none none none none none none none none none none none none none none none . , none none none none none none none

### <font color='blue'>PostLearning 9: filter low frequency words (optional)</font>
can be done only at the end of all data pipe generator, on the final data base TBD:
* replace low-frequent words with constant string (will do after the data set is ready)<br>
will be done after few iterations of learning

NameError: name 'genrate' is not defined